In [1]:
from Bio import pairwise2 as pw

from Bio import Seq

from Bio import SeqIO 
import Bio

from Bio.Alphabet import IUPAC

import numpy as np
import pandas as pd
import re

In [2]:
import time
from functools import wraps
  
def fn_timer(function):
  @wraps(function)
  def function_timer(*args, **kwargs):
    t0 = time.time()
    result = function(*args, **kwargs)
    t1 = time.time()
    print ("Total time running %s: %s seconds" %
        (function.__name__, str(t1-t0))
        )
    return result
  return function_timer

In [3]:
uniprot = list(SeqIO.parse('uniprot-proteome_human.fasta','fasta'))

In [4]:
for seq in uniprot:   #
    x = seq.id
    seq.id = x.split("|")[1]

In [5]:
ids = [seq.id for seq in uniprot]
names = [seq.name for seq in uniprot]

In [6]:
uni_dict = {}
for i in uniprot:
    uni_dict[i.id] =i

In [7]:
grist = pd.read_table('gristone_positive_data.txt')

In [8]:
grist.head(3)

,uniport_id,peptide,left_flanking,right_flanking,sample_id,ensembl_gene_id,express_sum_tpm,family,A1,B1,C1,A2,B2,C2,label
0,P62424,KVAPAPAVVKK,KAKGK,QEAKK,train_sample_28,ENSG00000148303,397.13,PTHR23105_SF29,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1
1,germline-TAOK3-NM_016281-140-G-to-A-S47N-Ser47...,HTNEVVAIKK,FATNA,MSYSG,train_sample_28,ENSG00000135090,21.64,PTHR24361_SF187,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1
2,Q460N5,RLLPGNATISK,VKREG,AGKLP,train_sample_28,ENSG00000173193,26.23,PTHR14453_SF53,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1


In [9]:
#@fn_timer
def calc_identity_score(seq1,seq2,gap=-0.5,extend=-0.1):
    """
    return seq1 seq1 pairwise identiy score
    seq1 is positive 
    seq1 seq2 is string

    Bio.pairwise2.format_alignment output:
    MPKGKKAKG------
      |||||||      
    --KGKKAKGKKVAPA
      Score=7
      
    alignment output: [('MPKGKKAKG------', '--KGKKAKGKKVAPA', 7.0, 0, 15)]
    score = ali[0][2] = 7
    """
    ali = pw.align.globalxs(seq1,seq2,gap,extend,score_only=True)
    # gap penalty = -0.5 in case of cak caak score =3
    return ali/min(len(seq1),len(seq2)) # 返回短序列的值,防止substring

In [10]:
#@fn_timer
def window_generator(seq, window_lenth=7,step=1):
    """
    return list of seq window slide
    seq is string
    """
    if len(seq) >= window_lenth:
        return [seq[i:i+window_lenth] for i in range(0,len(seq)-window_lenth+1,step)]
    else:
        return []

In [11]:
window_generator('123',3)

['123']

In [12]:
from Bio.pairwise2 import format_alignment

In [13]:
def flat(nums):
    res = []
    for i in nums:
        if isinstance(i, list):
            res.extend(flat(i))
        else:
            res.append(i)
    return res

In [14]:
#@fn_timer
def slide_with_flank(seq,full,step=1,up_flank=6,down_flank=6,flags=0):
    """
    return window slide result as list for a full str given potential sub seq and it's flank removed
    seq=abc, full = 01234abc45678abc1234 up=1 down=1 step =1
    result is ['012', '123', '567', '234']
    生成一个str full 的滑动窗口自片段,去除了seq和其上下游的部分
    """
    res = []
    window_len = len(seq)
    coords = [i.span() for i in re.finditer(seq,full,flags)] # = search_all
    # 处理首尾情况
    if len(coords) == 0:
        res.append(window_generator(full,window_lenth=window_len,step=step))

    elif len(coords) == 1:
        if (coords[0][0]-up_flank) >= 0:
            res.append(window_generator(full[0:coords[0][0]-up_flank],
                                        window_lenth=window_len,step=step))
        if (coords[0][1]+down_flank) <= len(full):
            res.append(window_generator(full[coords[0][1]+up_flank:],
                                        window_lenth=window_len,step=step))
    else: # len(coords) >1  
        if (coords[0][0]-up_flank) >= 0:
            res.append(window_generator(full[0:coords[0][0]-up_flank],
                                        window_lenth=window_len,step=step))
        for i in range(1,len(coords)):
        ## 处理 1 2 之间的东西
            if coords[i][0] - coords[i-1][1] > up_flank+down_flank+window_len:
                res.append(window_generator(full[coords[i-1][1]+up_flank:coords[i][0]-down_flank],
                                            window_lenth=window_len,step=step))
                           
        if (coords[-1][1]+down_flank) <= len(full):
            res.append(window_generator(full[coords[-1][1]+down_flank:],
                                        window_lenth=window_len,step=step))           
        
    return flat(res)

In [15]:
slide_with_flank('abc','01234abc456raa78abc7779',up_flank=2,down_flank=2)

['012', '6ra', 'raa']

In [16]:
###### @fn_timer
def filter_with_identity_affinity(seq,full,identity_cutoff=0.5,step=1,up=6,down=6,flags=0):
    filtered = {}
    slides = slide_with_flank(seq,full,step=step,up_flank=up,down_flank=down,flags=flags)
    for s in slides:
        identity_score = calc_identity_score(seq,s)
        if identity_score <= identity_cutoff:
            #if s in filtered.keys(): #已经有score, 寸大值表明有大片段
            filtered[s] = identity_score
    return filtered

In [ ]:
grist['neg1'] =np.full_like(grist['peptide'],'o')
t0 = time.process_time()
for i in range(0,len(grist)):
#for i in range(0,10000):
    g = grist.iloc[i]
    if g['uniport_id'] in ids:
        pep = str(g['left_flanking']) + str(g['peptide']) + str(g['right_flanking'])
        full = str(uni_dict[g['uniport_id']].seq)
        res_local = filter_with_identity_affinity(pep,full,step=10)
#        res_full = {}
#        for win in res_local.keys(): # 过滤掉阳性集在window结果中 identity>0.5的
#            for pep in np.random.choice(grist['peptide'],100):#.remove(g['peptide']):
#                score = calc_identity_score(win,pep)
#                if score <0.5:
#                    res_full[win] = score
#                else:
#                    break
#        res_full_key = sorted(res_full.keys(),key=lambda x:(res_local[x],res_full[x]))
        k=sorted(res_local.keys(),key=lambda x:res_local[x])
        if len(k):
            grist.loc[i,'neg1'] = k[0] 
        
print(time.process_time()-t0)
#grist[grist['neg1'] != 'o'].to_csv('testdata0401.txt',index = False)

In [18]:
grist[['left_flanking','right_flanking']]

,left_flanking,right_flanking
0,KAKGK,QEAKK
1,FATNA,MSYSG
2,VKREG,AGKLP
3,AALQA,SQAVK
4,PPPGM,RPPMG
...,...,...
246325,ISTAG,MLNGA
246326,KLSIM,GTLDS
246327,VTAPT,NSASG
246328,LFCYH,LMALY


In [28]:
a=grist['neg1'][0]
#gri

In [30]:
grist.loc?

Type:        property
String form: <property object at 0x7fced00e5b90>
Docstring:  
Access a group of rows and columns by label(s) or a boolean array.

``.loc[]`` is primarily label based, but may also be used with a
boolean array.

Allowed inputs are:

- A single label, e.g. ``5`` or ``'a'``, (note that ``5`` is
  interpreted as a *label* of the index, and **never** as an
  integer position along the index).
- A list or array of labels, e.g. ``['a', 'b', 'c']``.
- A slice object with labels, e.g. ``'a':'f'``.

  .. warning:: Note that contrary to usual python slices, **both** the
      start and the stop are included

- A boolean array of the same length as the axis being sliced,
  e.g. ``[True, False, True]``.
- A ``callable`` function with one argument (the calling Series or
  DataFrame) and that returns valid output for indexing (one of the above)

See more at :ref:`Selection by Label <indexing.label>`

Raises
------
KeyError:
    when any items are not found

See Also
--------
Dat

In [25]:
a

,uniport_id,peptide,left_flanking,right_flanking,sample_id,ensembl_gene_id,express_sum_tpm,family,A1,B1,C1,A2,B2,C2,label,neg1
neg1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,P62424,KVAPAPAVVKK,KAKGK,QEAKK,train_sample_28,ENSG00000148303,397.13,PTHR23105_SF29,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1.0,RTNYNDRYDEIRRHWGGNVLG


In [22]:
grist.loc?

Type:        property
String form: <property object at 0x7fced00e5b90>
Docstring:  
Access a group of rows and columns by label(s) or a boolean array.

``.loc[]`` is primarily label based, but may also be used with a
boolean array.

Allowed inputs are:

- A single label, e.g. ``5`` or ``'a'``, (note that ``5`` is
  interpreted as a *label* of the index, and **never** as an
  integer position along the index).
- A list or array of labels, e.g. ``['a', 'b', 'c']``.
- A slice object with labels, e.g. ``'a':'f'``.

  .. warning:: Note that contrary to usual python slices, **both** the
      start and the stop are included

- A boolean array of the same length as the axis being sliced,
  e.g. ``[True, False, True]``.
- A ``callable`` function with one argument (the calling Series or
  DataFrame) and that returns valid output for indexing (one of the above)

See more at :ref:`Selection by Label <indexing.label>`

Raises
------
KeyError:
    when any items are not found

See Also
--------
Dat

In [53]:
grist.head()

,uniport_id,peptide,left_flanking,right_flanking,sample_id,ensembl_gene_id,express_sum_tpm,family,A1,B1,C1,A2,B2,C2,label,neg1
0,P62424,KVAPAPAVVKK,KAKGK,QEAKK,train_sample_28,ENSG00000148303,397.13,PTHR23105_SF29,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1,PRYIRLQRQRA
1,germline-TAOK3-NM_016281-140-G-to-A-S47N-Ser47...,HTNEVVAIKK,FATNA,MSYSG,train_sample_28,ENSG00000135090,21.64,PTHR24361_SF187,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1,o
2,Q460N5,RLLPGNATISK,VKREG,AGKLP,train_sample_28,ENSG00000173193,26.23,PTHR14453_SF53,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1,o
3,O15511,ALKNPPINTK,AALQA,SQAVK,train_sample_28,ENSG00000162704,79.10,PTHR12644_SF1,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1,o
4,P09234,MPVGPAPGM,PPPGM,RPPMG,train_sample_28,ENSG00000124562,40.97,PTHR31148_SF1,A*03:01,B*35:03,C*03:03,A*03:01,B*51:01,C*04:01,1,o


In [22]:

persons={'ZhangSan':'male',
         'LiSi':'male',
         'WangHong':'female'}

#找出所有男性
males = filter(lambda x:'male'== x[1], persons.items())

for (key,value) in males:
    print('%s : %s' % (key,value))


ZhangSan : male
LiSi : male


In [42]:
grist.to_csv?

Signature:
grist.to_csv(
    path_or_buf=None,
    sep=',',
    na_rep='',
    float_format=None,
    columns=None,
    header=True,
    index=True,
    index_label=None,
    mode='w',
    encoding=None,
    compression='infer',
    quoting=None,
    quotechar='"',
    line_terminator=None,
    chunksize=None,
    date_format=None,
    doublequote=True,
    escapechar=None,
    decimal='.',
)
Docstring:
Write object to a comma-separated values (csv) file.

.. versionchanged:: 0.24.0
    The order of arguments for Series was changed.

Parameters
----------
path_or_buf : str or file handle, default None
    File path or object, if None is provided the result is returned as
    a string.  If a file object is passed it should be opened with
    `newline=''`, disabling universal newlines.

    .. versionchanged:: 0.24.0

       Was previously named "path" for Series.

sep : str, default ','
    String of length 1. Field delimiter for the output file.
na_rep : str, default ''
    Missing dat